## 0. 환경구성
### 0.1 패키지 Import


In [0]:
# 기초 함수 Import
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### 0.2 구글 드라이브 연결


In [2]:
!pip install kaggle

In [3]:
ls -1ha kaggle.json

kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 코드추가
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
# 데이터 적제
item_categories = pd.read_csv("/gdrive/My Drive/kaggle1/item_categories.csv")
items=pd.read_csv('/gdrive/My Drive/kaggle1/items.csv')
shops=pd.read_csv('/gdrive/My Drive/kaggle1/shops.csv')
test=pd.read_csv('/gdrive/My Drive/kaggle1/test.csv')
sp=pd.read_csv('/gdrive/My Drive/kaggle1/sample_submission.csv')
trn=pd.read_csv('/gdrive/My Drive/kaggle1/sales_train_v2.csv')

## 1. 데이터 전처리

### 1-1. 데이터 효율화

In [0]:
# 테스트에 있는 데이터만 선별
test_items_u = test.item_id.unique()
trn = trn[trn.item_id.isin(test_items_u)]

test_shops_u = shops.shop_id.unique()
trn = trn[trn.shop_id.isin(test_shops_u)]

### 1-2. 데이터 속성확인

In [8]:
# 속성 별 데이터 확인
# 각 열을 for 문을 통해서 출력
for col in trn.columns:
    print('{}\n'.format(trn[col].head()))
# '{}\n' 줄 띠어 쓰기를 의미.  

0     02.01.2013
10    03.01.2013
11    05.01.2013
12    07.01.2013
13    08.01.2013
Name: date, dtype: object

0     0
10    0
11    0
12    0
13    0
Name: date_block_num, dtype: int64

0     59
10    25
11    25
12    25
13    25
Name: shop_id, dtype: int64

0     22154
10     2574
11     2574
12     2574
13     2574
Name: item_id, dtype: int64

0     999.0
10    399.0
11    399.0
12    399.0
13    399.0
Name: item_price, dtype: float64

0     1.0
10    2.0
11    1.0
12    1.0
13    2.0
Name: item_cnt_day, dtype: float64



In [9]:
# 수치형 변수 확인하기

n_cols = [col for col in trn.columns[4:6]]
trn[n_cols].describe()

,item_price,item_cnt_day
count,1.446006e+06,1.446006e+06
mean,1.022199e+03,1.329992e+00
std,1.807847e+03,3.189945e+00
min,5.000000e-01,-1.600000e+01
25%,2.990000e+02,1.000000e+00
50%,5.490000e+02,1.000000e+00
75%,1.199000e+03,1.000000e+00
max,5.920000e+04,2.169000e+03


### 1-3. Input 형식 만들기

In [10]:
test.head()
# Test의 값에 ID, Shop_ID, Item_ID 가 있음으로, 이를 기준으로 형식으로 만들어야함.

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [0]:
# Month, Item ID, Shop ID 세개의 열을 이용해, 인풋 값을 만들자.

# 곱집합을 이용해, 빈 형식을 만들기
from itertools import product 

## Shop List와 Items List를 곱집합함
S_I_Prod = pd.DataFrame(list(product(test_shops_u, test_items_u)), columns=['shop_id','item_id']) 

# 모든 경우에 수에 알맞게, 트레인 값 병합
trn_B = pd.merge(S_I_Prod , trn, on=['shop_id','item_id'], how='left') 
trn_B['item_cnt_day'].fillna(0, inplace=True) 

# 그룹핑
trn_grp = trn_B.groupby(['shop_id','item_id'])

In [12]:
trn_B

,shop_id,item_id,date,date_block_num,item_price,item_cnt_day
0,0,5037,NaN,NaN,NaN,0.0
1,0,5320,NaN,NaN,NaN,0.0
2,0,5233,NaN,NaN,NaN,0.0
3,0,5232,NaN,NaN,NaN,0.0
4,0,5268,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
1617801,59,969,13.06.2014,17.0,549.0,1.0
1617802,59,969,25.06.2014,17.0,549.0,1.0
1617803,59,969,15.06.2014,17.0,549.0,1.0
1617804,59,969,12.07.2014,18.0,549.0,1.0


## 2. 파생변수 만들기

### 2.1 그룹 별 통계변수 추가

In [0]:
# 그룹별 합계, 빈도, 평균, 표준편차

trn_IC = pd.DataFrame(trn_grp.agg({'item_cnt_day':['sum','count','mean','std'],'item_price':['sum', 'count','mean', 'std']})).reset_index() 
#trn_IC.columns = ['date_block_num','shop_id','item_id','item_cnt_sum','item_cnt_count','item_cnt_mean','item_cnt_std''item_price_sum','item_price_count','item_price_mean','item_price_std'] 

In [0]:
trn_IC.columns = ['shop_id','item_id','item_cnt_sum','item_cnt_count','item_cnt_mean','item_cnt_std','item_price_sum','item_price_count','item_price_mean','item_price_std'] 

In [15]:
trn_IC.head()

,shop_id,item_id,item_cnt_sum,item_cnt_count,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std
0,0,30,31.0,9,3.444444,2.242271,2385.0,9,265.0,0.0
1,0,31,11.0,7,1.571429,0.975900,3038.0,7,434.0,0.0
2,0,32,16.0,11,1.454545,0.522233,2431.0,11,221.0,0.0
3,0,33,6.0,6,1.000000,0.000000,2082.0,6,347.0,0.0
4,0,38,0.0,1,0.000000,NaN,0.0,0,NaN,NaN


In [16]:
trn_IC['item_cnt_count'].describe()

count    306000.000000
mean          5.286948
std          16.646686
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         867.000000
Name: item_cnt_count, dtype: float64

### 2.2 아이탬, 상점 그룹 추가

In [17]:
pip install googletrans   #package 설치

In [0]:
# 러시아어 번역을 위한 함수
from googletrans import Translator

trans = Translator()
def translate(col):
    en_list=[]
    for word in col:
        en = trans.translate(word, src = 'ru', dest='en')   #러시아어(ru) -> 영어(en)로 번역
        en_list.append(en.text)   #Tranlator().text = 번역결과
    return en_list

In [0]:
# 10개만 선별
item_categories10 = item_categories.loc[1:10]
shop10 = shops.loc[1:10]

In [20]:
item_categories10['item_category_name'] = translate(item_categories10['item_category_name'])
shop10['shop_name'] = translate(shop10['shop_name'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
item_categories10
# 카테고리 이름의 첫 단어가 하위 항목을 포함하고 있음을 확인 가능

,item_category_name,item_category_id
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4
5,Accessories - PSVita,5
6,Accessories - XBOX 360,6
7,Accessories - XBOX ONE,7
8,Tickets (digits),8
9,Delivery of goods,9
10,Game consoles - PS2,10


In [22]:
shop10
# 상점 이름의 첫 단어가 지역명임을 확인 가능

,shop_name,shop_id
1,"! Yakutsk TC ""Central"" Franc",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TRC ""October-Kinomir""",3
4,"Volzhsky mall ""Volga Mall""",4
5,"Vologda SEC ""Marmalade""",5
6,"Voronezh (Plekhanovskaya, 13)",6
7,"Voronezh SEC ""Maksimir""",7
8,"Voronezh shopping center City Park ""Castle""",8
9,Itinerant trade,9
10,Zhukovsky Street. Chkalov 39m?,10


In [23]:
#카테고리 그룹 추가하기

# 카테고리의 처음이름 가져오기
item_grp = item_categories['item_category_name'].apply(lambda x: str(x).split(' ')[0]) 
# 코드 형식으로 바꿔 열 추가
item_categories['item_group'] = pd.Categorical(item_grp).codes 
# 아이탬에 그룹표기 추가
items = pd.merge(items, item_categories.loc[:,['item_category_id','item_group']], on=['item_category_id'], how='left') 
items.head()


,item_name,item_id,item_category_id,item_group
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,8
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,12
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,8
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,8
4,***КОРОБКА (СТЕКЛО) D,4,40,8


In [0]:
# 병합을 위한 테이블 생성
items_I = items[['item_id','item_category_id','item_group']]

In [25]:
#Shop에서 도시 추가하기

city = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0]) 
shops['city'] = pd.Categorical(city).codes 
shops.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,31
1,"! Yakutsk TC ""Central"" Franc",1,0
2,"Adygea TC ""Mega""",2,1
3,"Balashikha TRC ""October-Kinomir""",3,2
4,"Volzhsky mall ""Volga Mall""",4,5


In [0]:
# 병합을 위한 테이블 생성
shops_I = shops[['shop_id','city']]

### 2.3 파생 변수 합치기


In [0]:
# 아이탬 카테고리 합치기
trn_ICI = pd.merge(trn_IC, items_I, on=['item_id'])

In [0]:
# 상점 도시 합치기
trn_ICII = pd.merge(trn_IC, shops_I, on=['shop_id'])

In [29]:
trn_ICII.head()

,shop_id,item_id,item_cnt_sum,item_cnt_count,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std,city
0,0,30,31.0,9,3.444444,2.242271,2385.0,9,265.0,0.0,31
1,0,31,11.0,7,1.571429,0.975900,3038.0,7,434.0,0.0,31
2,0,32,16.0,11,1.454545,0.522233,2431.0,11,221.0,0.0,31
3,0,33,6.0,6,1.000000,0.000000,2082.0,6,347.0,0.0,31
4,0,38,0.0,1,0.000000,NaN,0.0,0,NaN,NaN,31


In [0]:
# 같은 정보를 담는 테스트 데이터 생성
# 모든 경우에 수에 알맞게, 트레인 값 병합
test_B = pd.merge(S_I_Prod, test, on=['shop_id','item_id'], how='left') 


In [31]:
test_B.head()

,shop_id,item_id,ID
0,0,5037,NaN
1,0,5320,NaN
2,0,5233,NaN
3,0,5232,NaN
4,0,5268,NaN


In [32]:
len(test)

214200

In [33]:
len(test_B)

306000

In [34]:
#r결측치 제거
test_B=test_B.dropna(axis=0)
len(test_B)

214200

In [0]:
# 아이템 카테고리 합치기
test_ICI = pd.merge(test_B, items_I, on=['item_id'], how= 'left')
# 상점 도시 합치기
test_ICII = pd.merge(test_B, shops_I, on=['shop_id'],how= 'left')

In [36]:
test_F = pd.merge(test_B, trn_IC, on=['shop_id','item_id'], how= 'left')
test_B['ID'].describe()

count    214200.000000
mean     107099.500000
std       61834.358168
min           0.000000
25%       53549.750000
50%      107099.500000
75%      160649.250000
max      214199.000000
Name: ID, dtype: float64

In [37]:
test_F.head()

,shop_id,item_id,ID,item_cnt_sum,item_cnt_count,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std
0,2,5037,20400.0,13.0,12,1.083333,0.288675,22288.5,12,1857.375000,636.971425
1,2,5320,20401.0,0.0,1,0.000000,NaN,0.0,0,NaN,NaN
2,2,5233,20402.0,18.0,17,1.058824,0.242536,14983.5,17,881.382353,308.668893
3,2,5232,20403.0,7.0,7,1.000000,0.000000,5394.0,7,770.571429,292.672517
4,2,5268,20404.0,0.0,1,0.000000,NaN,0.0,0,NaN,NaN


In [38]:
del test_F['item_cnt_count']
#예측해야하는 값이므로 제거함.
test_F.head()

,shop_id,item_id,ID,item_cnt_sum,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std
0,2,5037,20400.0,13.0,1.083333,0.288675,22288.5,12,1857.375000,636.971425
1,2,5320,20401.0,0.0,0.000000,NaN,0.0,0,NaN,NaN
2,2,5233,20402.0,18.0,1.058824,0.242536,14983.5,17,881.382353,308.668893
3,2,5232,20403.0,7.0,1.000000,0.000000,5394.0,7,770.571429,292.672517
4,2,5268,20404.0,0.0,0.000000,NaN,0.0,0,NaN,NaN


In [39]:
# 결측치는 0으로 변경
test_F.fillna(0, inplace=True)
test_B['ID'].describe()

count    214200.000000
mean     107099.500000
std       61834.358168
min           0.000000
25%       53549.750000
50%      107099.500000
75%      160649.250000
max      214199.000000
Name: ID, dtype: float64

## 3. 데이터 모델링

In [0]:
# 랜덤 포레스트를 이용
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [41]:
# 트리개수 선정
rfc = RandomForestClassifier(n_estimators=40)
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
# 학습을 위한 변수 설정

train = trn_ICII.copy()
train['item_cnt_count']

0          9
1          7
2         11
3          6
4          1
          ..
305995    16
305996     1
305997     6
305998     1
305999     4
Name: item_cnt_count, Length: 306000, dtype: int64

In [43]:
train_Y = train['item_cnt_count']
train_Y.fillna(0, inplace=True)
train_Y.describe()

count    306000.000000
mean          5.286948
std          16.646686
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         867.000000
Name: item_cnt_count, dtype: float64

In [44]:
train_X =trn_ICII.copy()
train_X.fillna(0, inplace=True)
train_X.head()
del train_X['item_cnt_count']
train_X.head()

,shop_id,item_id,item_cnt_sum,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std,city
0,0,30,31.0,3.444444,2.242271,2385.0,9,265.0,0.0,31
1,0,31,11.0,1.571429,0.975900,3038.0,7,434.0,0.0,31
2,0,32,16.0,1.454545,0.522233,2431.0,11,221.0,0.0,31
3,0,33,6.0,1.000000,0.000000,2082.0,6,347.0,0.0,31
4,0,38,0.0,0.000000,0.000000,0.0,0,0.0,0.0,31


In [45]:
# 모델 구축
rfc.fit(train_X,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
# 예측
 prediction=rfc.predict(test_F)

In [0]:
test_F["item_cnt_month"] = prediction

In [48]:
test_F.head()

,shop_id,item_id,ID,item_cnt_sum,item_cnt_mean,item_cnt_std,item_price_sum,item_price_count,item_price_mean,item_price_std,item_cnt_month
0,2,5037,20400.0,13.0,1.083333,0.288675,22288.5,12,1857.375000,636.971425,156
1,2,5320,20401.0,0.0,0.000000,0.000000,0.0,0,0.000000,0.000000,1
2,2,5233,20402.0,18.0,1.058824,0.242536,14983.5,17,881.382353,308.668893,156
3,2,5232,20403.0,7.0,1.000000,0.000000,5394.0,7,770.571429,292.672517,156
4,2,5268,20404.0,0.0,0.000000,0.000000,0.0,0,0.000000,0.000000,1


In [49]:
test_F["item_cnt_month"].describe()

count    214200.000000
mean         24.941545
std          70.592119
min           1.000000
25%           1.000000
50%           3.000000
75%           9.000000
max         867.000000
Name: item_cnt_month, dtype: float64

In [0]:
 Result = test_F[["ID","item_cnt_month"]]

In [51]:
Result.head()

,ID,item_cnt_month
0,20400.0,156
1,20401.0,1
2,20402.0,156
3,20403.0,156
4,20404.0,1


In [52]:
Result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 2 columns):
ID                214200 non-null float64
item_cnt_month    214200 non-null int64
dtypes: float64(1), int64(1)
memory usage: 4.9 MB


In [53]:
Result.ID = Result['ID'].astype('int')
Result.item_cnt_month = Result.item_cnt_month.astype('int')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
Result.to_csv('submission.csv',index=False)

In [55]:
#제출
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "Message"

100% 1.80M/1.80M [00:04<00:00, 407kB/s]
Successfully submitted to Predict Future Sales